In [38]:
import pandas as pd
import json

In [39]:
usernames = ["azlatkovic"]

In [41]:
usernames[0]

'azlatkovic'

In [55]:
import requests
import time
headers = {"Authorization": "Bearer XXXXX"}

def load_user(username):
    endpoint = f"https://oneapi.aol.com/one-central/user-management/v4/users/{username}?organizationId=0"
    result = requests.get(endpoint, headers=headers).json()
    # print(result)
    return result

def load_and_save_user(username):
    output_file = f"output/users/{username}.json"
    user = load_user(username)
    with open(output_file, 'w') as outfile:
        json.dump(user, outfile)

cnt = 0

username = "f8b858ca-aaa9-434d-a654-5b96b81b5cf8"
load_and_save_user(username)

In [46]:
def load_roles(username):
    endpoint = f"https://oneapi.aol.com/one-central/authorization-management/v2/entities/user/{username}/roles?type=functional&organizationId=0&limit=20&offset=0&order=asc&page=1&sort=name"
    result = requests.get(endpoint, headers=headers).json()
    # print(result)
    return result

# roles = load_roles("azlatkovic")
# roles

In [56]:
import glob
import pandas as pd

path = r'output/users/'
filenames = glob.glob(path + "/*.json")

users = []
for filename in filenames:
    users.append(pd.read_json(filename, orient='index'))

external_users = []
for cnt, user in enumerate(users):
    # print(user.loc(0)['scopes'][0])
    # print(f"{user[0].username} - {user[0].scopes}")
    try: 
        # print(user[0].internal)
        if not user[0].internal: 
            # print(f"{user[0].username} - {user[0].scopes}")
            external_users.append(user[0].username)
    except:
        print(f"Error ----------------- {filenames[cnt]}")
        # print(users[cnt])

# external_users

eu_df = pd.DataFrame(external_users, columns=['usernames'])

eu_df.head()

,usernames
0,7c7a3535-9202-485b-9639-b71acd0d6234
1,9b160335-3adb-4616-be6d-31326eecb016
2,95e12aaf-6ce9-448a-a232-63f248584a17
3,d07952f3-dc14-4261-8a0c-aeee4f2ced65
4,e889c061-643a-46cf-8b6d-70915f7e1f11


In [79]:
active_api_users_with_roles = pd.read_csv("../data/1c/1C_Api_Users_Active_In_Last30_With_Roles.csv", na_values=["(null)"])
active_api_users_with_roles

,username,name,application,display_name,internal
0,15939535-ecb6-4dd0-ae44-4f7e99578d6a,ATP readonly access,atp,ATP,1
1,82e32044-f622-4b71-96ab-305a23a02787,NaN,NaN,NaN,1
2,32e045d6-3435-45a2-8197-893a72f8a503,Vidible Video Org Test Acct - Do not delete - ...,NaN,NaN,1
3,f87dc6a4-0db7-4bc1-9688-e31c588c178f,ApiSeller,OneMobile,One Mobile,0
4,32e045d6-3435-45a2-8197-893a72f8a503,VSSP Audience Builder,dmp,DMP,1
...,...,...,...,...,...
56,c83c8441-0315-4652-b8f2-fb9458d01e2a,TagCacheUser,tagCache,TagCache,1
57,f67f7231-ffae-4539-8f1e-334596bfad1a,ONE Admin Read Only,oc,One Central,1
58,4a90923b-53ba-4ac9-b4e9-1039f4306b6f,ONE Admin Read Only,oc,One Central,1
59,32e045d6-3435-45a2-8197-893a72f8a503,ONE Admin Read Only,oc,One Central,1


In [95]:
work_df = active_api_users_with_roles[active_api_users_with_roles["display_name"].notna()]
work_df = work_df.drop(["name", "application", "internal"], axis=1)
work_df = work_df.rename(columns={"display_name":"platforms"})
# work_df
grouped_df = work_df.groupby("username")["platforms"].agg(list).reindex()
grouped_df

username
04b432b3-6ff3-45f0-b388-0839c3d7ccd6                              [One Central]
15939535-ecb6-4dd0-ae44-4f7e99578d6a     [ATP, Aol Store, Aol Store, Aol Store]
195bb12e-a363-4810-90e3-0acc3b2b7e8f                              [One Central]
1f97b067-c299-4436-80bb-cb5d6898e0d9    [One Central, One Central, One Central]
21272183-c01a-4fdd-aee0-ffd9f22aa976                              [One Central]
2d829a91-3394-4db9-ab4a-63f8754a2b3c                              [One Central]
2dfec50f-b1fa-46af-8f03-eb875a1609f9                  [One Mobile, One Central]
32e045d6-3435-45a2-8197-893a72f8a503                         [DMP, One Central]
4a90923b-53ba-4ac9-b4e9-1039f4306b6f                 [One Central, One Central]
96f9ca02-c4b3-4186-80db-fbf7e68b51ac                 [One Central, One Central]
9f4f2b6f-0011-4fdd-91d3-4b6e26b08adf                   [Aol Store, One Central]
ae1ce2ca-686d-45fb-a16e-c42fbb8d4957                   [One Central, Aol Store]
b3236ffd-038e-4dde-8530-a18e647

In [99]:
active_api_users = pd.read_csv("../data/1c/2024-10-15-Users - ActiveApiUsersLast30Days.csv")
# active_api_users

pd.merge(active_api_users, grouped_df, on=["username"], how="left")

,username,Status,Internal,last_login_date,platforms
0,32e045d6-3435-45a2-8197-893a72f8a503,ACTIVE,True,2024-10-15 10:22:16,"[DMP, One Central]"
1,15939535-ecb6-4dd0-ae44-4f7e99578d6a,ACTIVE,True,2024-10-15 10:22:15,"[ATP, Aol Store, Aol Store, Aol Store]"
2,82e32044-f622-4b71-96ab-305a23a02787,ACTIVE,True,2024-10-15 10:22:06,NaN
3,96f9ca02-c4b3-4186-80db-fbf7e68b51ac,ACTIVE,True,2024-10-15 10:21:55,"[One Central, One Central]"
4,d67e9d4c-7b42-41d3-bded-b0e6edc02e46,ACTIVE,True,2024-10-15 10:20:41,"[One Central, TagCache]"
5,c95f30ec-8407-4d4a-9377-49ded38997a7,ACTIVE,True,2024-10-15 10:15:53,"[Aol Store, Aol Store, One Central]"
6,ca92a9ad-dde9-4e68-be4a-460b656c66a7,ACTIVE,True,2024-10-15 10:07:39,"[One Audience, TagCache]"
7,c83c8441-0315-4652-b8f2-fb9458d01e2a,ACTIVE,True,2024-10-15 10:04:31,"[One Central, TagCache]"
8,1f97b067-c299-4436-80bb-cb5d6898e0d9,ACTIVE,True,2024-10-15 9:27:44,"[One Central, One Central, One Central]"
9,21272183-c01a-4fdd-aee0-ffd9f22aa976,Pending Activation,True,2024-10-15 9:26:13,[One Central]


In [128]:
def merge_users_and_platforms(up_file, u_file, res_file):
    updf = pd.read_csv(up_file, na_values=["(null)"])
    
    updf = updf[updf["display_name"].notna()]
    updf = updf.drop(["name", "application", "internal"], axis=1)
    updf = updf.rename(columns={"display_name":"platforms"})
    updf = updf.groupby("username")["platforms"].agg(list).reindex()
    
    
    udf = pd.read_csv(u_file)
    
    udf = pd.merge(udf, updf, on=["username"], how="left")
    
    # udf.head()
    # udf.index.size
    
    udf.to_csv(res_file)

In [129]:
user_platforms_file = "../data/1c/1C_Users_Active_In_Last30_With_Roles.csv"
users_file = "../data/1c/2024-10-15-Users - ActiveUsersLast30Days.csv"
user_res_file = "./output/active_30d_with_platform.csv"

merge_users_and_platforms(user_platforms_file, users_file, user_res_file)

In [130]:
user_platforms_file = "../data/1c/1C_OIDC_Users_Active_With_Roles.csv"
users_file = "../data/1c/2024-10-15-Users - OIDC Users.csv"
user_res_file = "./output/active_oidc_with_platform.csv"


merge_users_and_platforms(user_platforms_file, users_file, user_res_file)